11.2.3 Neumann Laplace Indirect Method
======================================================
**keys**: homogeneous Neumann bvp, hypersingular operator

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

Consider the Neumann boundary value problem 

$$ \left\{ \begin{array}{rlc l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_1 u&=& u_1, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

The solution $u\in H^1(\Omega)$ of the above bvp can be written in the following form (representation forumula) 

$$ u(x) =  \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{ \langle n(y), x-y \rangle }{\| x-y\|^3} } \, m(y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{DL}(m) }}\,.$$ 

Let's carefully apply the Neumann trace $\gamma_1$ to the representation formula and solve the resulting boundary integral equation for $m \in H^{\frac12}(\Gamma)$ by discretisation of the following variational formulation: 

$$ \forall \, v\in H^{\frac12}(\Gamma): \left\langle v, \gamma_1 \left(\mathrm{DL}(m)\right) \right\rangle_{-\frac12} = \left\langle u_1, v\right\rangle_{-\frac12} \,. $$

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)

Define the finite element space for $H^{\frac12}(\Gamma)$ for given geometry :  

In [ ]:
fesH1 = H1(mesh, order=2, definedon=mesh.Boundaries(".*"))
uH1,vH1 = fesH1.TnT()
print ("ndof H1 = ", fesH1.ndof)

Define the right hand side with given Neumann data $u_1$:

In [ ]:
uexa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
graduexa = CF( (uexa.Diff(x), uexa.Diff(y), uexa.Diff(z)) )

n = specialcf.normal(3)
u1exa = graduexa*n
rhs = LinearForm(u1exa*vH1.Trace()*ds(bonus_intorder=3)).Assemble()

The discretisation of the variational formulation leads to a system of linear equations, ie 

$$ \left( \mathrm{D} + \mathrm{S}\right) \, \mathrm{m}= \mathrm{rhs}\,, $$ 

where $\mathrm{D}$ is the hypersingular operator and the stabilisation $(\mathrm D + \mathrm{S})$ is regular and symmetric.

In [ ]:
vH1m1 = LinearForm(vH1*1*ds(bonus_intorder=3)).Assemble()
S = (BaseMatrix(Matrix(vH1m1.vec.Reshape(1))))@(BaseMatrix(Matrix(vH1m1.vec.Reshape(fesH1.ndof))))

In [ ]:
m = GridFunction(fesH1)
pre = BilinearForm(uH1*vH1*ds).Assemble().mat.Inverse(freedofs=fesH1.FreeDofs()) 
with TaskManager(): 
    D=HypersingularOperator(fesH1, intorder=12)
    CG(mat = D.mat+S, pre=pre, rhs = rhs.vec, sol=m.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)

Draw (m, mesh, draw_vol=False, order=3);

**Note:** the hypersingular operator is implemented as follows 

$$ D = \langle v, \gamma_1 \mathrm{DL}(m) \rangle_{-\frac12} = \frac{1}{4\pi} \int\limits_\Gamma\int\limits_\Gamma \frac{ \langle \mathrm{\boldsymbol {curl}}_\Gamma  \,m(x), \mathrm{\boldsymbol{curl}}_\Gamma\, v(y) \rangle}{\|x-y\|} \, \mathrm{d} \sigma_{y} \, \mathrm{d} \sigma_x $$

Details for instance in [Numerische Näherungsverfahren für elliptische Randwertprobleme](https://link.springer.com/book/10.1007/978-3-322-80054-1), p.127, p.259 (1st edition).